In [3]:
import pickle
import ml.structures
import ml.dataset.corpus as corpus
import numpy as np

import pickle
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV

from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import RMSprop


from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.engine.topology import Input

from sklearn.metrics import accuracy_score
from keras.layers import Concatenate, Merge

import keras
from keras.utils import to_categorical

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [4]:
def make_16(chord):
    c = 16 / (128 / chord.duration)
    array = np.array([note.number for note in chord.notes])
    if len(array) == 0:
        array = [0]
    if c >= 1:
        return 1, np.tile(array, (int(c), 1))
    else:
        return 0, [0]

In [5]:
def cut_song(song, notes, chords):
    chords_track = []
    notes_track = np.array([])
    flag = 0
    for note in song.tracks[0].chords:
        if (make_16(note)[0] == 0):
            flag = 1
        notes_track = np.append(notes_track, make_16(note)[1])
    for chord in song.tracks[1].chords:
        if (make_16(chord)[0] == 0):
            flag = 1
        chords_track.append(make_16(chord)[1])
    if flag == 1:
        return 0, notes, chords
    return 1, notes_track, chords_track

In [6]:
def norm_array(X, size):
    for j in range(len(X)):
        array = np.zeros(size-len(X[j])) - 1
        X[j] = np.array(X[j])
        X[j] = np.append(X[j], array)
    X = np.array(X)
    return X

In [7]:
def create_X_Y(chords):
    real_chords = []
    y = []
    for i, subchord in enumerate(chords):
        if i != len(chords) - 1:
            for ssubchord in subchord:
                real_chords.append(ssubchord)
            size = len(subchord)
            for j in range(size):
                y.append(chords[i+1][-1])
    return real_chords, y

In [8]:
def notes_to_np(notes):
    for i in range(len(notes)):
        notes[i] = np.array(notes[i])
    notes = np.array(notes)

In [9]:
def create_unique_chords(X, unique_chords):
    for x in X:
        unique_chords = np.append(unique_chords, np.array(x))
    return unique_chords

In [10]:
def create_coded(X):
    return np.zeros(len(X))


In [11]:
def encode(X, coded_X):
    for j, k in enumerate(X):
        for i, unique_chord in enumerate(unique_chords):
            if np.array_equal(k, unique_chord):
                coded_X[j] = i

In [62]:
model_chords = Sequential()
model_notes = Sequential()

model_chords.add(Embedding(10000, 20, input_length=1, batch_input_shape=(1,1)))
model_notes.add(Embedding(128, 20, input_length=1, batch_input_shape=(1,1)))
merged_model = Sequential()
merged_model.add(Merge([model_chords, model_notes], mode="concat"))
merged_model.add(LSTM(20, stateful=True))
merged_model.add(Dense(10000, activation='softmax'))
merged_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


unique_chords = np.array([])

with open('gtp_dataset_final.pickle', 'rb') as input_file:
    i = 0
    while True:
        try:
            i += 1
            song = ml.structures.Song()
            song.undump(input_file)
            chords = []
            notes = []
            flag_notes_chords = cut_song(song, notes, chords)
            if (flag_notes_chords[0] == 0) :
                continue
            notes = flag_notes_chords[1]
            chords = flag_notes_chords[2]
            X = []
            Y = []
            X, Y = create_X_Y(chords)
            X = norm_array(X, 15)
            Y = norm_array(Y, 15)
            notes_to_np(notes)
            unique_chords = create_unique_chords(X, unique_chords)
            unique_chords = create_unique_chords(Y, unique_chords)
            unique_chords = unique_chords.reshape((-1, 15))
            unique_chords = np.unique(unique_chords, axis=0)
            unique_chords = list(unique_chords)
            coded_X = []
            coded_Y = []
            coded_X = create_coded(X)
            coded_Y = create_coded(Y)
            encode(X, coded_X)
            encode(Y, coded_Y)
            X = np.array(coded_X)
            y = np.array(coded_Y)
            X_notes = notes
            X_notes = X_notes[:len(X)]
            X = X[:len(X_notes)]
            y = y[:len(X)]
            if (i < 100):
                merged_model.fit([X, X_notes], y, epochs=1, batch_size=1, verbose=1)
                merged_model.reset_states()
            else:
                break
                merged_model.predict([X, X_notes], batch_size=1, verbose=1)
            
            
        except EOFError:
            break

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  import sys


Epoch 1/1
446/446 [==============================] - 7s 15ms/step - loss: 4.5225 - acc: 0.1794
Epoch 1/1
286/286 [==============================] - 4s 15ms/step - loss: 2.5222 - acc: 0.1783
Epoch 1/1
90/90 [==============================] - 1s 13ms/step - loss: 3.2636 - acc: 0.0222
Epoch 1/1
90/90 [==============================] - 1s 12ms/step - loss: 2.5649 - acc: 0.2444
Epoch 1/1
94/94 [==============================] - 1s 13ms/step - loss: 5.8455 - acc: 0.0957
Epoch 1/1
94/94 [==============================] - 1s 13ms/step - loss: 4.3357 - acc: 0.3830
Epoch 1/1
114/114 [==============================] - 2s 14ms/step - loss: 6.1361 - acc: 0.1404
Epoch 1/1
94/94 [==============================] - 1s 15ms/step - loss: 4.9685 - acc: 0.1277
Epoch 1/1
94/94 [==============================] - 1s 14ms/step - loss: 3.4739 - acc: 0.2553
Epoch 1/1
644/644 [==============================] - 9s 14ms/step - loss: 6.5475 - acc: 0.0295
Epoch 1/1
516/516 [==============================] - 7s 13ms/s

254/254 [==============================] - 3s 14ms/step - loss: 4.5055 - acc: 0.1614
Epoch 1/1
60/60 [==============================] - 1s 13ms/step - loss: 12.2178 - acc: 0.0000e+00
Epoch 1/1
274/274 [==============================] - 4s 13ms/step - loss: 7.7393 - acc: 0.0547
Epoch 1/1
240/240 [==============================] - 3s 13ms/step - loss: 6.4391 - acc: 0.1333
Epoch 1/1
116/116 [==============================] - 1s 13ms/step - loss: 3.7995 - acc: 0.3879
Epoch 1/1
302/302 [==============================] - 4s 13ms/step - loss: 2.6579 - acc: 0.3543
Epoch 1/1
260/260 [==============================] - 3s 13ms/step - loss: 1.7270 - acc: 0.4769
Epoch 1/1
134/134 [==============================] - 2s 12ms/step - loss: 10.5038 - acc: 0.0075
Epoch 1/1
126/126 [==============================] - 2s 12ms/step - loss: 9.8179 - acc: 0.0000e+00
Epoch 1/1
254/254 [==============================] - 3s 12ms/step - loss: 10.5293 - acc: 0.0000e+00


In [44]:
import pandas as pd

df = pd.DataFrame(unique_chords)

In [45]:
df.to_csv("vocab.csv")

In [63]:
x = pd.read_csv("vocab.csv")

In [57]:
x = x.values.T[1:].T

In [58]:
x

array([[ 0., -1., -1., ..., -1., -1., -1.],
       [25., 49., -1., ..., -1., -1., -1.],
       [25., 61., -1., ..., -1., -1., -1.],
       ...,
       [75., 54., 61., ..., -1., -1., -1.],
       [76., 49., -1., ..., -1., -1., -1.],
       [76., 61., -1., ..., -1., -1., -1.]])

In [ ]:
merged_model.save("NN_model.h5")